In [1]:
%matplotlib inline

import numpy as np
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import random as rd
import json
import itertools
import re
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.text as kpt
from keras.models import load_model, model_from_json
import pickle

# loading
# with open('/root/w210_capstone/data/models/tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)
# # load json and create model
# json_file = open('/root/w210_capstone/data/models/model_simple_nn.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights('/root/w210_capstone/data/models/model_simple_nn_WEIGHTS.h5')
# # NDB_tag_unique_unique_dict
with open('/root/w210_capstone/data/models/NDB_tag_unique_unique_dict.json') as f:
    NDB_tag_unique_unique_dict = json.load(f)

# load recipes
with open('/root/w210_capstone/data/recipe/recipe_clean.json') as f:
    recipe_clean = json.load(f)
    
# nutritional database
nutritional_database = pd.read_csv('/root/w210_capstone/data/nutrient/compiled/nutrition_master_df.csv')

ambiguous = pd.read_csv("/root/w210_capstone/data/recipe/ambiguous_ingredients_from_recipes.csv")
ambiguous = ambiguous.drop_duplicates(subset='JSON_ValueString')

height has been deprecated.



Using TensorFlow backend.


In [2]:
units_of_food_recipe = ["cups", "cup", "c.", "c", 
                        "fl. oz.", "fl oz", "fluid ounce", "fluid ounces", "fl_oz",
                        "gal", "gal.", "gallon", "gallons",
                        "oz", "oz.", "ounce", "ounces", "ouncs",
                        "pt", "pt.", "pint", "pints",
                        "lb", "lb.", "pound", "pounds",
                        "qt", "qt.", "qts", "qts.", "quart", "quarts",
                        "tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs", "TBSP",
                        "tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons",
                        "g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes",
                        "kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes",
                        "l", "l.", "liter", "liters", "litre", "litres",
                        "mg", "mg.", "milligram", "milligrams", "milligramme", "milligrammes",
                        "ml", "ml.", "milliliter", "milliliters", "millilitre", "millilitres",
                        "pinch", "pinches", "dash", "dashes", "touch", "touches", "handful", "handfuls",
                        "stick", "sticks",
                        "cans", "can",
                        "to taste",
                        "scoop", "scoops",
                        "dollop", "dollops",
                        "sprig", "sprigs",
                       "recipe",
                       "garnish", "garnished",
                       "sprinkle", "spinkled",
                       "ribs", "rib", "stalk",
                       "inch", "inches", "in.",
                       "drizzle", "drizzled",
                       "to taste"]


def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
def optimize_with_single_n(possible_list, target_nutrs):
    
    recipe_combination = list(itertools.combinations(possible_list, 1))
    cleaned_combination = [x[0] for x in recipe_combination]
    
    print(recipe_combination)
    
    best_loss = {'carb': 1000, 'protein': 1000, 'fat': 1000}
    for iteration in cleaned_combination:
        
        recipe = possible_list[iteration]
        current_carb, current_protein, current_fat = 0, 0, 0
        
        next_carb = current_carb + recipe['carb']
        next_protein = current_protein + recipe['protein']
        next_fat = current_fat + recipe['fat']
        
        if next_carb > target_nutrs['carb'] or next_protein > target_nutrs['protein'] or next_fat > target_nutrs['fat']:
            print('This recipe does not fit the target nutrients')
        
        count = 1
        while next_carb <= target_nutrs['carb'] and next_protein <= target_nutrs['protein'] and next_fat <= target_nutrs['fat']:
            current_carb = next_carb
            current_protein = next_protein
            current_fat = next_fat
            
            next_carb = current_carb + recipe['carb']
            next_protein = current_protein + recipe['protein']
            next_fat = current_fat + recipe['fat']
            count += 1
    
    
        carb_loss = str(target_nutrs['carb'] - current_carb)
        protein_loss = str(target_nutrs['protein'] - current_protein)
        fat_loss = str(target_nutrs['fat'] - current_fat)
        
        
        print('For '+str(count)+" servings of "+iteration+" carb is "+str(current_carb)+", protein is "+str(current_protein)+", fat is "+str(current_fat)+".")
        print('Loss is '+carb_loss+' carbs, '+protein_loss+' protein, '+fat_loss+' fat.')
        print('\n')

In [4]:
def macro_daily_macro_estimation(profile_dict):
    
    profile_dict['macro_nutrients'] = {}
    
    height = profile_dict['characteristics']['height']
    weight = profile_dict['characteristics']['weight']
    age = profile_dict['characteristics']['age']
    calorie_factor = profile_dict['factors']['calorie_factor']
    protein_factor = profile_dict['factors']['protein_factor']
    
    
    # The Original Harris-Benedict Equation   
    calories = 66.473 + 13.7516 * (weight / 2.2)  + 5.0033 * (height * 2.54) - 6.755 * age    
    calories = calories * calorie_factor
    
    ## The Revised Harris-Benedict Equation
    #calories = 88.362 + 13.397 * (weight * 2.2) + 4.799 * (height * 2.54) - 5.6777 * age
    ## The Mifflin St Jeor Equation
    #calories = 5 + 10 * (weight * 2.2) + 6.25 * (height * 2.54) - 5 * age

    # Proteins (g)
    protein = weight * protein_factor
    
    # Fats (g)
    fat = (calories * 0.2) / 9.0
    
    # Carbohydrates (g)
    carbohydrates = (calories - ((protein * 4.0) + (fat * 9.0))) / 4.0
    
    # Fiber (g)
    fiber = (calories / 1000.0) * 14.0
    
    # Cholesterol (mg)
    cholesterol = 300
    
    # Saturated Fat (g)
    saturated_fat = (calories * 0.1 ) / 9.0
    
    # Unsaturated Fat (g)
    unsaturated_fat = (calories * 0.3) / 9.0
    
    # Sugar (g)
    sugar = (calories * 0.1 ) / 9.0
    
    profile_dict['macro_nutrients']['calories'] = calories
    profile_dict['macro_nutrients']['fat'] = fat
    profile_dict['macro_nutrients']['carbohydrates'] = carbohydrates
    profile_dict['macro_nutrients']['fiber'] = fiber
    profile_dict['macro_nutrients']['cholesterol'] = cholesterol
    profile_dict['macro_nutrients']['saturated_fat'] = saturated_fat
    profile_dict['macro_nutrients']['unsaturated_fat'] = unsaturated_fat
    profile_dict['macro_nutrients']['sugar'] = sugar
    profile_dict['macro_nutrients']['protein'] = protein
    
    return profile_dict

In [5]:
def conversion_rate(ingredient_amount, ingredient_unit, nutrition_db_amount, nutrition_db_unit, nutrition_grams):
    
    key_dict = {"cup": {"cups", "cup", "c.", "c"},
        "fluid_ounce": {"fl. oz.", "fl oz", "fluid ounce", "fluid ounces", "fl_oz"},
        "gallon": {"gal", "gal.", "gallon", "gallons"},
        "ounce": {"oz", "oz.", "ounce", "ounces", "ouncs"},
        "pint": {"pt", "pt.", "pint", "pints"},
        "pound": {"lb", "lb.", "pound", "pounds"},
        "quart": {"qt", "qt.", "qts", "qts.", "quart", "quarts"},
        "tablespoon": {"tbsp.", "tbsp", "T", "T.", "tablespoon", "tablespoons", "tbs.", "tbs", "TBSP"},
        "teaspoon": {"tsp.", "tsp", "t", "t.", "teaspoon", "teaspoons"},
        "gram": {"g", "g.", "gr", "gr.", "gram", "grams", "gramme", "grammes"},
        "kilogram": {"kg", "kg.", "kilogram", "kilograms", "kilogramme", "kilogrammes"}
        }
    
    dictionary_conversions = {
        'tablespoon': {
            'teaspoon': 3.0,
            'tablespoon': 1.0,
            'cup': 0.0616115
        },
        'teaspoon': {
            'teaspoon': 1.0,
            'tablespoon': 0.333,
            'cup': 0.0205372         
        },
        'cup': {
            'teaspoon': 48.692115,
            'tablespoon': 16.2307,
            'cup': 1.0,
            'fluid_ounce': 8.11537
        },
        
        'pound': {
            'gram': 16.0,
            'pound': 1.0,
            'kilogram': 0.453592,
            'gram': 453.592
        },
        'ounce': {
            'ounce': 1.0,
            'pound': 0.0625,
            'kilogram': 0.0283495,
            'gram': 28.3495
            
        },
        'kilogram': {
            'ounce': 35.274,
            'pound': 0.0625,
            'kilogram': 2.20462,
            'gram': 1000.0
        },
        'fluid_ounce': {
            'cup': 0.123223
        }
    }    
    
    for key in key_dict.keys():
        if ingredient_unit in key_dict[key]:
            ingredient_unit = key
        if nutrition_db_unit in key_dict[key]:
            nutrition_db_unit = key

    if ingredient_unit == 'pound' or ingredient_unit == 'ounce' or ingredient_unit == 'kilogram':
        nutrition_db_unit = 'gram'
            
    temp_conversation_rate = dictionary_conversions[ingredient_unit][nutrition_db_unit]
            
    if ingredient_unit == 'pound' or ingredient_unit == 'ounce' or ingredient_unit == 'kilogram':
        conversion_rate = float(temp_conversation_rate) * float(ingredient_amount)
    else:
        conversion_rate = float(temp_conversation_rate) * (float(ingredient_amount) / float(nutrition_db_amount)) * float(nutrition_grams)
    
    conversion_rate = float(conversion_rate) / float(nutrition_grams)
    
    return conversion_rate

In [6]:
def food_info_extraction(units_of_food_recipe, nutritional_database, ingredient, nbd_tag): 
    temp_nutrition = nutritional_database[nutritional_database['NDB_NO'] ==  "\"{}\"".format(nbd_tag.strip('"'))][['NDB_NO', 'Measure', 'Weight(g)', ]]    
    temp_nutrition_measure = temp_nutrition['Measure'].get_values()[0]
    
    if 'fl oz' in temp_nutrition_measure:
        temp_nutrition_measure = temp_nutrition_measure.replace('fl oz', 'fl_oz')    
    temp_nutrition_measure = temp_nutrition_measure.replace(',', '')
    temp_nutrition_measure_split = temp_nutrition_measure.split(" ")
    temp_nutrition_grams = temp_nutrition['Weight(g)'].get_values()[0]
    
    nutrition_db_unit_list = list(set(temp_nutrition_measure_split).intersection(units_of_food_recipe))  

    if len(nutrition_db_unit_list) == 1:
        nutrition_db_unit_use = nutrition_db_unit_list[0]
        temp_nutrition_measure_index = temp_nutrition_measure_split.index(nutrition_db_unit_use)          
        if is_number(temp_nutrition_measure_split[temp_nutrition_measure_index - 1]):
            nutrition_db_amount_use = float(temp_nutrition_measure_split[temp_nutrition_measure_index - 1])
            
    ingredient = ingredient.replace(',', '')
    ingredient_split = ingredient.split(" ")
    ingredient_split_unit_list = list(set(ingredient_split).intersection(units_of_food_recipe))
    
    if len(nutrition_db_unit_list) == 1:
        ingredient_split_unit_use = ingredient_split_unit_list[0]
        ingredient_split_unit_index = ingredient_split.index(ingredient_split_unit_use) 
                
        if is_number(temp_nutrition_measure_split[ingredient_split_unit_index - 1]):
            ingredient_amount_use = float(ingredient_split[ingredient_split_unit_index - 1])
    
    return ingredient_amount_use, ingredient_split_unit_use, nutrition_db_amount_use, nutrition_db_unit_use, temp_nutrition_grams

In [7]:
def ambiguous_coversion_factor(grams, nutritional_database, nbd_tag):
    
    try:
        temp_nutrition = nutritional_database[nutritional_database['NDB_NO'] ==  "\"{}\"".format(nbd_tag.strip('"'))][['NDB_NO', 'Measure', 'Weight(g)', ]]    
    except:
        temp_nutrition = nutritional_database[nutritional_database['NDB_NO'] ==  "\"{}\"".format(nbd_tag)][['NDB_NO', 'Measure', 'Weight(g)', ]]    
    
    temp_nutrition_measure = temp_nutrition['Weight(g)'].get_values()[0]
    
    return float(grams) / float(temp_nutrition_measure)
    

In [8]:
# recipe_id_list = list(recipe_clean.keys())[:1]
recipe_id_list = list(recipe_clean.keys())

recipe_itr = 0
for recipe_id in recipe_id_list:
    print(recipe_id)
    conversion_rate_factor_list = []    
    itr = 0
    for tag in recipe_clean[recipe_id]['NDB_NO_tags']:
        if tag != 'np.nan' and tag != '':            
            split_ingredient = recipe_clean[recipe_id]['ingredients'][itr].split(" ")
            split_ingredient_filter = list(filter(None, split_ingredient))
            ingredient_new_temp = " ".join(split_ingredient_filter)
            
            if ingredient_new_temp.strip("'") in ambiguous['JSON_ValueString'].values:               
                conversion_rate_factor = ambiguous_coversion_factor(ambiguous[ambiguous['JSON_ValueString'] == ingredient_new_temp.strip("'")]['grams'].get_values()[0], nutritional_database, tag) 
                conversion_rate_factor_list.append(conversion_rate_factor)
            else:                
                try:
                    ingredient_amount_use, ingredient_split_unit_use, nutrition_db_amount_use, nutrition_db_unit_use, temp_nutrition_grams = food_info_extraction(units_of_food_recipe, 
                                         nutritional_database, 
                                         ingredient_new_temp, 
                                         tag)
                    conversion_rate_factor = conversion_rate(ingredient_amount_use, ingredient_split_unit_use, nutrition_db_amount_use, nutrition_db_unit_use, temp_nutrition_grams)
                    conversion_rate_factor_list.append(conversion_rate_factor)
                except:
                    print("\t**********FAILED!**********")
                    print(recipe_id)
                    print(ingredient_new_temp)
        else:
            conversion_rate_factor_list.append('')
        itr += 1
        recipe_clean[recipe_id]['conversion_rate_factor'] = conversion_rate_factor_list
    recipe_itr += 1

RECIPE_76881
RECIPE_69129
RECIPE_78461
RECIPE_78505
RECIPE_49004
RECIPE_67528
RECIPE_41736
RECIPE_12111
RECIPE_24578
RECIPE_13188
RECIPE_18440
RECIPE_71831
RECIPE_63893
RECIPE_44107
RECIPE_8669
RECIPE_52892
RECIPE_84387
RECIPE_71937
RECIPE_21388
RECIPE_14063
RECIPE_61308
RECIPE_83991
RECIPE_69249
RECIPE_45110
RECIPE_65290
RECIPE_62881
RECIPE_61302
RECIPE_60641
RECIPE_63775
RECIPE_49066
RECIPE_67146
RECIPE_55944
RECIPE_41612
RECIPE_63951
RECIPE_67225
RECIPE_72219
RECIPE_69809
RECIPE_78967
RECIPE_25618
RECIPE_51469
RECIPE_80454
RECIPE_67456
RECIPE_12451
RECIPE_24481
RECIPE_66587
RECIPE_80100
RECIPE_10144
RECIPE_83998
RECIPE_71531
RECIPE_72342
RECIPE_72825
RECIPE_59307
RECIPE_78922
RECIPE_52818
RECIPE_56992
RECIPE_45775
RECIPE_24182
RECIPE_73684
RECIPE_66924
RECIPE_11555
	**********FAILED!**********
RECIPE_11555
4.5 pounds pork ribs
RECIPE_16039
RECIPE_79845
RECIPE_75566
RECIPE_47829
RECIPE_46184
RECIPE_81528
RECIPE_54934
RECIPE_43613
RECIPE_73604
RECIPE_15171
RECIPE_13046
RECIPE_7817
REC

In [9]:
macro_list = \
['Energy (kcal)', 'Total lipid (fat) (g)', 'Carbohydrate, by difference (g)', 
'Fiber, total dietary (g)', 'Cholesterol (mg)', 
'Fatty acids, total saturated (g)',
'Fatty acids, total monounsaturated (g)', 'Fatty acids, total polyunsaturated (g)', 
 'Fatty acids, total trans (g)', 'Protein (g)', 'Sugars, total (g)']

micro_list = \
['Iron, Fe (mg)', 'Magnesium, Mg (mg)', 'Manganese, Mn (mg)', 'Thiamin (mg)', 
 'Vitamin D (D2 + D3) (µg)']

keep_list = ['NDB_NO', 'Measure', 'Weight(g)', 'Description', 'Category']

macro_micro_sum_list = macro_list + micro_list
macro_micro_list = keep_list + macro_list + micro_list
ndb_columns_filtered = nutritional_database[macro_micro_list]

In [10]:
# recipe_id_list = list(recipe_clean.keys())[:1]
recipe_id_list = list(recipe_clean.keys())

recipe_itr = 0
for recipe_id in recipe_id_list:
    print(recipe_id)
    print(recipe_clean[recipe_id]['name'])
    itr = 0
    df_list = []
    for tag in recipe_clean[recipe_id]['NDB_NO_tags']:
        if tag != 'np.nan' and tag != '':
            df_list.append(ndb_columns_filtered[ndb_columns_filtered['NDB_NO'] ==  "\"{}\"".format(tag.strip('"'))][macro_micro_sum_list])
        itr += 1
    master_df = pd.concat(df_list).sum()
    
    recipe_clean[recipe_id]['carb'] = master_df['Carbohydrate, by difference (g)']
    recipe_clean[recipe_id]['fat'] = master_df['Total lipid (fat) (g)']
    recipe_clean[recipe_id]['protein'] = master_df['Protein (g)']

RECIPE_76881
Shrimp Potstickers
RECIPE_69129
Avocado Beet Pea and Cucumber Salad
RECIPE_78461
Fried Green Tomatoes
RECIPE_78505
Parmesan Kale Chips
RECIPE_49004
Jades BBP (Breakfast Bread Pudding)
RECIPE_67528
Tuscan Chicken Pizza Sandwich
RECIPE_41736
Frittata with Tuna and Tomatoes
RECIPE_12111
Butter Pie Crust Dough
RECIPE_24578
Chicken Soup with Stars and Meatballs
RECIPE_13188
Rum-Raisin Pound Cake
RECIPE_18440
Super Potato Salad
RECIPE_71831
Peeky-Toe Crab and Green Tomato Salad with Chilled Gazpacho Sauce
RECIPE_63893
Chorizo Puffs (BunUelos de Chorizo)
RECIPE_44107
Honey Whole-Wheat Pancakes with Honey Lime Butter
RECIPE_8669
Roast Chicken with Dried Fruit and Almonds
RECIPE_52892
Individual Peach Tarte Tatin
RECIPE_84387
Champagne and Berry Soup
RECIPE_71937
Spanish Potato Salad
RECIPE_21388
Ruby Red Margarita
RECIPE_14063
Grilled New York Steaks with San Marzano Sauce
RECIPE_61308
Mediterranean Tuna Wrap
RECIPE_83991
Fritatten Soup with Romaine
RECIPE_69249
Endive and Grape S

RECIPE_9967
Chocolate-Cashew Toffee
RECIPE_66083
Spaghetti with Endives and Anchovies
RECIPE_9829
Tortellini with Porcini Mushroom Sauce
RECIPE_22479
Chipotle Mashed Potatoes
RECIPE_14116
Bread Crumbs for Pasta
RECIPE_72176
Fruit Salad with Lemon-Lime Dressing
RECIPE_82679
Baked Popcorn Rice
RECIPE_56307
Hot Punch (Ponche)
RECIPE_61036
Potato and Smoked Salmon Kugel
RECIPE_69078
Chino Farms Mixed Bean Salad with Mint Vinaigrette
RECIPE_43864
Smores Cupcakes
RECIPE_14937
Peperoncini Stuffed with Smoked Salmon and Dill Cream
RECIPE_16028
Pinto Bean Sausage and Fennel Gratin
RECIPE_63506
Roasted Cauliflower Melt
RECIPE_81365
Easy Cheesy Fondue with Fingerling Potatoes French Bread and Select Vegetables
RECIPE_16114
Roasted Sweet-Potato Spears with Bacon Vinaigrette
RECIPE_72950
Teriyaki Salmon with Mirin Cucumber Salad
RECIPE_49299
Shortcut Shepherds Pie
RECIPE_14602
Provencal Fish Stew with Rouille
RECIPE_84343
Tuscan Bean Soup with Greens
RECIPE_8521
Coconut Cupcakes with White Chocolat

In [11]:
mikegruzynski = {
    
    "characteristics":{
        'height': 76.0,
        'weight': 255.0,
        'age': 28.0
        },
    "factors":{
        'calorie_factor': 1.375,
        'protein_factor': 0.75
    },
    "micro_nutrients":{
        'iron': 8,
        'magnesium': 420,
        'maganese': 2.3,
        'thiamin': 1.2,
        'Vitamin D (D2 + D3)': 15        
    }
    
}

mikegruzynski = macro_daily_macro_estimation(mikegruzynski)
weekly_macro_nutrients = dict([(x, mikegruzynski['macro_nutrients'][x]*5) for x in mikegruzynski['macro_nutrients'].keys()])

In [33]:
recipes = {}
for recipe_id in recipe_clean.keys():
    recipes[recipe_id] = {'carb': recipe_clean[recipe_id]['carb'], 
                          'protein': recipe_clean[recipe_id]['protein'], 
                          'fat': recipe_clean[recipe_id]['fat']}
weekly_macro_nutrients_use = {'carb': weekly_macro_nutrients['carbohydrates'], 
                              'fat': weekly_macro_nutrients['fat'], 
                              'protein': weekly_macro_nutrients['protein']}

In [34]:
current_nutrients = {'carb': 0, 'protein': 0, 'fat': 0}
total_loss = weekly_macro_nutrients

In [35]:
optimize_with_single_n(recipes, weekly_macro_nutrients_use)

[('RECIPE_76881',), ('RECIPE_69129',), ('RECIPE_78461',), ('RECIPE_78505',), ('RECIPE_49004',), ('RECIPE_67528',), ('RECIPE_41736',), ('RECIPE_12111',), ('RECIPE_24578',), ('RECIPE_13188',), ('RECIPE_18440',), ('RECIPE_71831',), ('RECIPE_63893',), ('RECIPE_44107',), ('RECIPE_8669',), ('RECIPE_52892',), ('RECIPE_84387',), ('RECIPE_71937',), ('RECIPE_21388',), ('RECIPE_14063',), ('RECIPE_61308',), ('RECIPE_83991',), ('RECIPE_69249',), ('RECIPE_45110',), ('RECIPE_65290',), ('RECIPE_62881',), ('RECIPE_61302',), ('RECIPE_60641',), ('RECIPE_63775',), ('RECIPE_49066',), ('RECIPE_67146',), ('RECIPE_55944',), ('RECIPE_41612',), ('RECIPE_63951',), ('RECIPE_67225',), ('RECIPE_72219',), ('RECIPE_69809',), ('RECIPE_78967',), ('RECIPE_25618',), ('RECIPE_51469',), ('RECIPE_80454',), ('RECIPE_67456',), ('RECIPE_12451',), ('RECIPE_24481',), ('RECIPE_66587',), ('RECIPE_80100',), ('RECIPE_10144',), ('RECIPE_83998',), ('RECIPE_71531',), ('RECIPE_72342',), ('RECIPE_72825',), ('RECIPE_59307',), ('RECIPE_789

For 19 servings of RECIPE_84243 carb is 2297.16, protein is 0.0, fat is 49.5.
Loss is 97.610044 carbs, 956.25 protein, 322.835560444 fat.


For 35 servings of RECIPE_80009 carb is 2390.2, protein is 11.56, fat is 185.3.
Loss is 4.570044 carbs, 944.69 protein, 187.035560444 fat.


For 79 servings of RECIPE_6841 carb is 1422.72, protein is 63.96, fat is 370.5.
Loss is 972.050044 carbs, 892.29 protein, 1.83556044444 fat.


For 5 servings of RECIPE_73630 carb is 691.84, protein is 0.36, fat is 309.4.
Loss is 1702.930044 carbs, 955.89 protein, 62.9355604444 fat.


For 8 servings of RECIPE_61117 carb is 2222.85, protein is 10.08, fat is 81.2.
Loss is 171.920044 carbs, 946.17 protein, 291.135560444 fat.


For 17 servings of RECIPE_44942 carb is 2280.64, protein is 0.0, fat is 287.52.
Loss is 114.130044 carbs, 956.25 protein, 84.8155604444 fat.


For 4 servings of RECIPE_8520 carb is 588.24, protein is 0.78, fat is 370.38.
Loss is 1806.530044 carbs, 955.47 protein, 1.95556044444 fat.


For 28 

In [36]:
def optimize_with_multiple_n(possible_list, target_nutrs, number_recipes, minimum = 0):
    
    recipe_combination = list(itertools.combinations(possible_list, number_recipes))
    cleaned_combination = recipe_combination
    
    counter = 0
    best_loss = sum(target_nutrs.values())
    best_recipes = None
    best_weights = None
    
    for iteration in cleaned_combination:
        
        print("Optimizing combination "+str(counter+1))
        print("\n")
        macro_vector = [(possible_list[x]['carb'], possible_list[x]['protein'], possible_list[x]['fat']) for x in iteration]
        
        # define objective function as total macro loss with minimization
        c = [sum(target_nutrs.values())]+[-sum(x) for x in macro_vector]
        
        # define constraints, add 0 to account for constant
        carb_const = [0]+[x[0] for x in macro_vector]
        protien_const = [0]+[x[1] for x in macro_vector]
        fat_const = [0]+[x[2] for x in macro_vector]
        A = [carb_const, protien_const, fat_const]
        
        # define inequalities
        b = target_nutrs.values()
        
        # define bounds, note x0 must be 1 to define minimization
        # might want minimum of each
        x0_bounds = (1, 1)
        var_bounds = [x0_bounds]+[(minimum, None) for x in range(len(iteration))]

        
        #run optimization
        res = linprog(c, A_ub=A, b_ub=list(b), bounds=var_bounds, options={"disp": False})        
                
        if res['fun'] < best_loss:
            
            print('The new best loss is '+str(round(res['fun'], 4)))
            print('\n')
            best_loss = res['fun']
        
            best_recipes = iteration

            best_weights = res['x'][1:]
        counter = counter + 1
        
    return best_recipes, best_weights

In [37]:
final_recipes, weights = optimize_with_multiple_n(recipes, weekly_macro_nutrients_use, 2)

Optimizing combination 1


The new best loss is 365.6734


Optimizing combination 2


Optimizing combination 3


Optimizing combination 4


Optimizing combination 5


Optimizing combination 6


The new best loss is 364.614


Optimizing combination 7


Optimizing combination 8


Optimizing combination 9


Optimizing combination 10


Optimizing combination 11


Optimizing combination 12


Optimizing combination 13


Optimizing combination 14


Optimizing combination 15


Optimizing combination 16


Optimizing combination 17


Optimizing combination 18


Optimizing combination 19


Optimizing combination 20


Optimizing combination 21


Optimizing combination 22


Optimizing combination 23


Optimizing combination 24


The new best loss is 121.2186


Optimizing combination 25


Optimizing combination 26


Optimizing combination 27


Optimizing combination 28


Optimizing combination 29


Optimizing combination 30


Optimizing combination 31


Optimizing combination 32


Optimizing combina



Optimizing combination 286


Optimizing combination 287


Optimizing combination 288


Optimizing combination 289


Optimizing combination 290


Optimizing combination 291


Optimizing combination 292


Optimizing combination 293


Optimizing combination 294


Optimizing combination 295


Optimizing combination 296


Optimizing combination 297


Optimizing combination 298


Optimizing combination 299


Optimizing combination 300


Optimizing combination 301


Optimizing combination 302


Optimizing combination 303


Optimizing combination 304


Optimizing combination 305


Optimizing combination 306


Optimizing combination 307


The new best loss is 35.5131


Optimizing combination 308


Optimizing combination 309


Optimizing combination 310


Optimizing combination 311


Optimizing combination 312


Optimizing combination 313


Optimizing combination 314


Optimizing combination 315


Optimizing combination 316


Optimizing combination 317


Optimizing combination 318


Optimizing



Optimizing combination 586


Optimizing combination 587


Optimizing combination 588


Optimizing combination 589


Optimizing combination 590


Optimizing combination 591


Optimizing combination 592


Optimizing combination 593


Optimizing combination 594


Optimizing combination 595


Optimizing combination 596


Optimizing combination 597


Optimizing combination 598


Optimizing combination 599


Optimizing combination 600


Optimizing combination 601


Optimizing combination 602


Optimizing combination 603


Optimizing combination 604


Optimizing combination 605


Optimizing combination 606


Optimizing combination 607


Optimizing combination 608


Optimizing combination 609


Optimizing combination 610


Optimizing combination 611


Optimizing combination 612


Optimizing combination 613


Optimizing combination 614


Optimizing combination 615


Optimizing combination 616


Optimizing combination 617


Optimizing combination 618


Optimizing combination 619


Optimizing c

Optimizing combination 930


Optimizing combination 931


Optimizing combination 932


Optimizing combination 933


Optimizing combination 934


Optimizing combination 935


Optimizing combination 936


Optimizing combination 937


Optimizing combination 938


Optimizing combination 939


Optimizing combination 940


Optimizing combination 941


Optimizing combination 942


Optimizing combination 943


Optimizing combination 944


Optimizing combination 945


Optimizing combination 946


Optimizing combination 947


Optimizing combination 948


Optimizing combination 949


Optimizing combination 950


Optimizing combination 951


Optimizing combination 952


Optimizing combination 953


Optimizing combination 954


Optimizing combination 955


Optimizing combination 956


Optimizing combination 957


Optimizing combination 958


Optimizing combination 959


Optimizing combination 960


Optimizing combination 961


Optimizing combination 962


Optimizing combination 963


Optimizing com

Optimizing combination 1255


Optimizing combination 1256


Optimizing combination 1257


Optimizing combination 1258


Optimizing combination 1259


Optimizing combination 1260


Optimizing combination 1261


Optimizing combination 1262


Optimizing combination 1263


Optimizing combination 1264


Optimizing combination 1265


Optimizing combination 1266


Optimizing combination 1267


Optimizing combination 1268


Optimizing combination 1269


Optimizing combination 1270


Optimizing combination 1271


Optimizing combination 1272


Optimizing combination 1273


Optimizing combination 1274


Optimizing combination 1275


Optimizing combination 1276


Optimizing combination 1277


Optimizing combination 1278


Optimizing combination 1279


Optimizing combination 1280


Optimizing combination 1281


Optimizing combination 1282


Optimizing combination 1283


Optimizing combination 1284


Optimizing combination 1285


Optimizing combination 1286


Optimizing combination 1287


Optimizing

...



Optimizing combination 43580


Optimizing combination 43581


Optimizing combination 43582


Optimizing combination 43583


Optimizing combination 43584


Optimizing combination 43585


Optimizing combination 43586


Optimizing combination 43587


Optimizing combination 43588


Optimizing combination 43589


Optimizing combination 43590


Optimizing combination 43591


Optimizing combination 43592


Optimizing combination 43593


Optimizing combination 43594


Optimizing combination 43595


Optimizing combination 43596


Optimizing combination 43597


Optimizing combination 43598


Optimizing combination 43599


Optimizing combination 43600


Optimizing combination 43601


Optimizing combination 43602


Optimizing combination 43603


Optimizing combination 43604


Optimizing combination 43605


Optimizing combination 43606


Optimizing combination 43607


Optimizing combination 43608


Optimizing combination 43609


Optimizing combination 43610


Optimizing combination 43611


Optimi



Optimizing combination 44057


Optimizing combination 44058


Optimizing combination 44059


Optimizing combination 44060


Optimizing combination 44061


Optimizing combination 44062


Optimizing combination 44063


Optimizing combination 44064


Optimizing combination 44065


Optimizing combination 44066


Optimizing combination 44067


Optimizing combination 44068


Optimizing combination 44069


Optimizing combination 44070


Optimizing combination 44071


Optimizing combination 44072


Optimizing combination 44073


Optimizing combination 44074


Optimizing combination 44075


Optimizing combination 44076


Optimizing combination 44077


Optimizing combination 44078


Optimizing combination 44079


Optimizing combination 44080


Optimizing combination 44081


Optimizing combination 44082


Optimizing combination 44083


Optimizing combination 44084


Optimizing combination 44085


Optimizing combination 44086


Optimizing combination 44087


Optimizing combination 44088


Optimi

Optimizing combination 44454


Optimizing combination 44455


Optimizing combination 44456


Optimizing combination 44457


Optimizing combination 44458


Optimizing combination 44459


Optimizing combination 44460


Optimizing combination 44461


Optimizing combination 44462


Optimizing combination 44463


Optimizing combination 44464


Optimizing combination 44465


Optimizing combination 44466


Optimizing combination 44467


Optimizing combination 44468


Optimizing combination 44469


Optimizing combination 44470


Optimizing combination 44471


Optimizing combination 44472


Optimizing combination 44473


Optimizing combination 44474


Optimizing combination 44475


Optimizing combination 44476


Optimizing combination 44477


Optimizing combination 44478


Optimizing combination 44479


Optimizing combination 44480


Optimizing combination 44481


Optimizing combination 44482


Optimizing combination 44483


Optimizing combination 44484


Optimizing combination 44485


Optimizi

Optimizing combination 44787


Optimizing combination 44788


Optimizing combination 44789


Optimizing combination 44790


Optimizing combination 44791


Optimizing combination 44792


Optimizing combination 44793


Optimizing combination 44794


Optimizing combination 44795


Optimizing combination 44796


Optimizing combination 44797


Optimizing combination 44798


Optimizing combination 44799


Optimizing combination 44800


Optimizing combination 44801


Optimizing combination 44802


Optimizing combination 44803


Optimizing combination 44804


Optimizing combination 44805


Optimizing combination 44806


Optimizing combination 44807


Optimizing combination 44808


Optimizing combination 44809


Optimizing combination 44810


Optimizing combination 44811


Optimizing combination 44812


Optimizing combination 44813


Optimizing combination 44814


Optimizing combination 44815


Optimizing combination 44816


Optimizing combination 44817


Optimizing combination 44818


Optimizi

Optimizing combination 45115


Optimizing combination 45116


Optimizing combination 45117


Optimizing combination 45118


Optimizing combination 45119


Optimizing combination 45120


Optimizing combination 45121


Optimizing combination 45122


Optimizing combination 45123


Optimizing combination 45124


Optimizing combination 45125


Optimizing combination 45126


Optimizing combination 45127


Optimizing combination 45128


Optimizing combination 45129


Optimizing combination 45130


Optimizing combination 45131


Optimizing combination 45132


Optimizing combination 45133


Optimizing combination 45134


Optimizing combination 45135


Optimizing combination 45136


Optimizing combination 45137


Optimizing combination 45138


Optimizing combination 45139


Optimizing combination 45140


Optimizing combination 45141


Optimizing combination 45142


Optimizing combination 45143


Optimizing combination 45144


Optimizing combination 45145


Optimizing combination 45146


Optimizi

In [38]:
final_recipes, weights

(('RECIPE_53521', 'RECIPE_42860'), array([  0.        ,  18.09695492]))